# Run Backtesting on Historical Data for Amazon Lookout for Metrics

Amazon Lookout for Metrics also supports backtesting against your historical information and in this notebook we will demonstrate this functionality on the same e-commerce data. Once the backtesting job has completed you can see all of the anomalies that Amazon Lookout for Metrics detected in the last 20% of your historical data. From here you can begin to unpack the kinds of results you will see from Amazon Lookout for Metrics in the future when you start streaming in new data. **NOTE YOU MUST CREATE A NEW DETECTOR TO LEVERAGE REAL TIME DATA. BACKTESTING IS ONLY FOR EXPLORATION.**

This notebook assumes that you already completed the prerequisites in the `1.PrereqSetupPackages.ipynb` and `2.PrereqSetupData.ipynb`. If you have not, go back and complete those first, then give backtesting a whirl while you are waiting for the real time anomalies to alert you!

First restore the variables from the previous notebook and then import the libraries needed.

In [ ]:
%store -r

Just as in the last notebook, connect to AWS through the SDK.

In [ ]:
import boto3
region = "us-west-2"
session = boto3.Session(region_name = region)

# FIXME : Beta endpoint
L4M = session.client( "lookoutmetrics", endpoint_url='https://lookoutmetrics-beta.us-west-2.amazonaws.com/' )

## Creating A Detector

This step is identical to the one in the previous notebook. Here, we're simply creating a diffrent `detector` that we will use for backtesting. 

In [ ]:
project = "initial-poirot-backtesting"

response = L4M.create_anomaly_detector( 
    AnomalyDetectorName = project + "-detector",
    # AnomalyDetectorDomain = "ADS",
    AnomalyDetectorDescription = "My Detector",
    AnomalyDetectorConfig = {
        "AnomalyDetectorFrequency" : "PT1H",
    },
)

anomaly_detector_arn = response["AnomalyDetectorArn"]
anomaly_detector_arn

## Define Metrics

After creating a detector, we need to point it to the s3 path for our backtest data. This process is also, similar to the one from the previous notebook.

First, let's create a role that can work with the Amazon Lookout for Metrics service:

In [ ]:
from utility import get_or_create_iam_role
role_arn = get_or_create_iam_role(role_name)

Now, let's create a metric set for our detector that point to the backtest data in S3:

In [ ]:
s3_path_backtest = 's3://'+ s3_bucket + '/ecommerce/backtest/'
s3_path_backtest

In [ ]:
params = {
    "AnomalyDetectorArn": anomaly_detector_arn,
    "MetricSetName" : project + '-metric-set-1',
    "MetricList" : [
        {
            "MetricName" : "views",
            "AggregationFunction" : "AVG",
        },
        {
            "MetricName" : "revenue",
            "AggregationFunction" : "SUM",
        },
    ],

    "DimensionList" : [ "platform", "marketplace" ],

    "TimestampColumn" : {
        "ColumnName" : "timestamp",
        "ColumnFormat" : "yyyy-MM-dd HH:mm:ss",
    },

   #"Delay" : 120, # seconds the detector will wait before attempting to read latest data per current time and detection frequency below
    "MetricSetFrequency" : frequency,

    "MetricSource" : {
        "S3SourceConfig": {
            "RoleArn" : role_arn,
            "HistoricalDataPathList": [
                s3_path_backtest,
            ],
#            "TemplatedPathList": [
#                s3_path_format,
#            ],

            "FileFormatDescriptor" : {
                "CsvFormatDescriptor" : {
                    "FileCompression" : "NONE",
                    "Charset" : "UTF-8",
                    "ContainsHeader" : True,
                    "Delimiter" : ",",
#                    "HeaderList" : [
#                        "platform",
#                        "marketplace",
#                        "timestamp",
#                        "views",
#                        "revenue"
#                    ],
                    "QuoteSymbol" : '"'
                },
            }
        }
    },
}

params

In [ ]:
response = L4M.create_metric_set( ** params )

metric_set_arn = response["MetricSetArn"]
metric_set_arn

## Activate the Detector

Now that the MetricSet has been specified, we are ready to start backtesting, that's done by activating the back test anomaly detector.

In [ ]:
L4M.back_test_anomaly_detector(AnomalyDetectorArn = anomaly_detector_arn)

In [ ]:
from utility import wait_anomaly_detector
wait_anomaly_detector( L4M, anomaly_detector_arn )

## Validate results

After backtesting is finished, you can visually validate the historical anomalies via the console or inspect the results by running the following commands:

In [ ]:
response = L4M.list_anomaly_group_summaries(AnomalyDetectorArn = anomaly_detector_arn,
                                 SensitivityThreshold=50,
                                 MaxResults=10,
                                )
response

And to dive even deeper into a specific anomaly group, simlpy choose your anomaly group of interest and drill down to it's time-series.

In [ ]:
response = L4M.list_anomaly_group_time_series(AnomalyDetectorArn = anomaly_detector_arn,
                                   AnomalyGroupId='c1db0cc9-ec45-4467-ba4d-2459eec86d68',
                                   MetricName='views',
                                   MaxResults=10,
                                  )
response